In [ ]:
!pip install tensorflow keras scikit-learn nltk

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords

from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Dense
from tensorflow.keras.layers import Flatten, GlobalMaxPooling1D
from tensorflow.keras.layers import Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
import keras

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
import kagglehub

path = kagglehub.dataset_download("lakshmi25npathi/imdb-dataset-of-50k-movie-reviews")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'imdb-dataset-of-50k-movie-reviews' dataset.
Path to dataset files: /kaggle/input/imdb-dataset-of-50k-movie-reviews


In [ ]:
df = pd.read_csv(f"../kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv")
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [ ]:
df["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


## Data Preprocessing

In [ ]:
!pip install contractions

In [ ]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import contractions

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('punkt_tab')
stop_words = set(stopwords.words('english'))

def preprocess_text(text, use_lemmatization=False, use_stemming=False):

    text = text.lower()

    text = BeautifulSoup(text, "html.parser").get_text()

    text = contractions.fix(text)

    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'\d+', '', text)

    text = text.translate(str.maketrans('', '', string.punctuation))

    words = nltk.word_tokenize(text)
    words = [w for w in words if w not in stop_words]

    if use_lemmatization:
        lemmatizer = WordNetLemmatizer()
        words = [lemmatizer.lemmatize(w) for w in words]
    elif use_stemming:
        stemmer = PorterStemmer()
        words = [stemmer.stem(w) for w in words]

    words = [w for w in words if len(w) > 1]

    clean_text = " ".join(words)

    return clean_text


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [ ]:
df["review"] = df["review"].apply(preprocess_text)
df["review"]

,review
0,one reviewers mentioned watching oz episode ho...
1,wonderful little production filming technique ...
2,thought wonderful way spend time hot summer we...
3,basically family little boy jake thinks zombie...
4,petter matteis love time money visually stunni...
...,...
49995,thought movie right good job creative original...
49996,bad plot bad dialogue bad acting idiotic direc...
49997,catholic taught parochial elementary schools n...
49998,going disagree previous comment side maltin on...


In [ ]:
df["sentiment"] = df["sentiment"].apply(lambda x: 1 if x == "positive" else 0)

In [ ]:
df.head()

,review,sentiment
0,one reviewers mentioned watching oz episode ho...,1
1,wonderful little production filming technique ...,1
2,thought wonderful way spend time hot summer we...,1
3,basically family little boy jake thinks zombie...,0
4,petter matteis love time money visually stunni...,1


## Word2Vec

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df["review"], df["sentiment"], test_size=0.2, random_state=42)

In [ ]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
vocab_size = len(tokenizer.word_index) + 1

maxlen = 100

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [ ]:
import kagglehub

# Download latest version
path_glove = kagglehub.dataset_download("danielwillgeorge/glove6b100dtxt")

print("Path to dataset files:", path_glove)

Using Colab cache for faster access to the 'glove6b100dtxt' dataset.
Path to dataset files: /kaggle/input/glove6b100dtxt


In [ ]:
embeddings_dict = {}

glove_file = open(f"../kaggle/input/glove6b100dtxt/glove.6B.100d.txt")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = np.asarray(records[1:], dtype='float32')
    embeddings_dict[word] = vector_dimensions
glove_file.close()

In [ ]:
embeddings_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_dict.get(word)
    if embedding_vector is not None:
        embeddings_matrix[i] = embedding_vector

## Model WorkFlow

In [ ]:
from keras.layers import LSTM, Input

model = Sequential([
    Input(shape=(maxlen,)),
    Embedding(vocab_size, 100, weights=[embeddings_matrix], trainable=False),
    LSTM(128, return_sequences=True),
    Dropout(0.2),
    LSTM(64),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ (None, 100, 100)       │    18,553,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ (None, 100, 128)       │       117,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 100, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_3 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,720,521 (71.41 MB)

 Trainable params: 166,721 (651.25 KB)

 Non-trainable params: 18,553,800 (70.78 MB)

In [ ]:
model.compile(optimizer=keras.optimizers.AdamW(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
from keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=5)

In [ ]:
epochs = 25
batch_size = 128
X_train = X_train.astype('float32')
y_train = y_train.astype('float32')

history = model.fit(
    X_train,
    y_train,
    verbose=1,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping]
)

Epoch 1/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 8s 18ms/step - accuracy: 0.6809 - loss: 0.5832 - val_accuracy: 0.8139 - val_loss: 0.4250
Epoch 2/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.8200 - loss: 0.4096 - val_accuracy: 0.8377 - val_loss: 0.3974
Epoch 3/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.8445 - loss: 0.3596 - val_accuracy: 0.8239 - val_loss: 0.4100
Epoch 4/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.8556 - loss: 0.3346 - val_accuracy: 0.8591 - val_loss: 0.3253
Epoch 5/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.8681 - loss: 0.3106 - val_accuracy: 0.8625 - val_loss: 0.3180
Epoch 6/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.8782 - loss: 0.2950 - val_accuracy: 0.8651 - val_loss: 0.3226
Epoch 7/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 16ms/step - accuracy: 0.8885 - loss: 0.2741 - val_accuracy: 0.8717 - val_loss: 0.3013
Epoch 8/25
313/313 ━━━━━━━━━━━━━━━━━━━━ 5s 17ms/step - accuracy: 0.8941 - loss: 0.2652 - val_accu

In [ ]:
score = model.evaluate(X_test, y_test)
print("Test Score:", score[0])
print("Test Accuracy:", score[1])

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8705 - loss: 0.3491
Test Score: 0.34757131338119507
Test Accuracy: 0.8705000281333923


In [ ]:
instance = df["review"][60]
instance

'happened basically solid plausible premise decent talented cast somewhere movie loses actually never really got going little excitement find angie really pregnant find steve martin talented person usually brings lot movie dreadful entire character even close important movie make longer really would liked see interactions main characters kate angie maybe try pure comedy unfortunately maybe drama comedic elements think movie could funny since actresses quite funny ways sitting think numerous scenarios would riot'

In [ ]:
instance = tokenizer.texts_to_sequences([instance])
instance = pad_sequences(instance, padding='post', maxlen=maxlen)

model.predict(instance)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 236ms/step


array([[0.01109408]], dtype=float32)

In [ ]:
instance_in = input("Enter Your Review: ")
instance_in = preprocess_text(instance_in)
instance_in = tokenizer.texts_to_sequences([instance_in])
instance_in = pad_sequences(instance_in, padding='post', maxlen=maxlen)

prediction = model.predict(instance_in)

if prediction > 0.94:
    print("Positive")
    print(prediction)
else:
    print("Negative")
    print(prediction)

Enter Your Review: the movie wasn't enjoyable although it was hard cored and a good cast but bad experience
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Negative
[[0.24286957]]


In [ ]:
model.save("sent-analysis.keras")